In [9]:
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username='allenjeep', api_key='oEqxD0sQYCbdEn7VOnfi')
from plotly.graph_objs import *
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import random

author_id=pickle.load(open("F:/dataminin/nips-papers/new/index_id.dat", "rb"))
author_name=pickle.load(open("F:/dataminin/nips-papers/new/index_name.dat", "rb"))
co_author = pickle.load(open("F:/dataminin/nips-papers/new/raw_co_author.dat", "rb"))
cluster_author = pickle.load(open("F:/dataminin/nips-papers/new/raw_cluster_author.dat", "rb"))
connect=pickle.load(open("F:/dataminin/nips-papers/new/pagerank_graph4.dat", "rb"))
a=cluster_author["cluster4"] #change cluster name 0--6
G= nx.Graph()
G.add_weighted_edges_from(connect)
for i in range(len(a)):
    if np.all(co_author[author_id.index(a[i])]==0):
        G.add_node(a[i])
d={}
for key in  a:
    d[key]=[random.uniform(0,1),random.uniform(0,1)]

nx.set_node_attributes(G,'pos',d)
pos=nx.get_node_attributes(G,'pos')

dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d


p=nx.single_source_shortest_path_length(G,ncenter)

edge_trace = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)
    
storekey=[]
for key in G.node:
    storekey.append(str(key))
storename=[]
for i in range(len(a)):
    storename.append(author_name[author_id.index(int(storekey[i]))])
for node, adjacencies in enumerate(G.adjacency_list()):
    node_trace['marker']['color'].append(len(adjacencies))
    node_info =storekey[node]+storename[node]+'#connection: '+str(len(adjacencies))
    node_trace['text'].append(node_info)

fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br>Cluster4',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict( 
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='cluster4')